In [2]:
from scipy import sparse
import numpy as np
import time
import scipy as sp


In [19]:
class fm_tool(object):
    def __init__(self,epochs=5,learningRate=0.01):
        self.epochs = epochs
        self.learningRate = learningRate

        
    def data_type(self,data):
        if not type(data) is sp.sparse.csr.csr_matrix:
            return sparse.csr_matrix(data)
        else:
            return data
        
        
    def predict_raw(self,trainX):
        #result = (trainX.T*self.weight_first).sum(axis=0).reshape(trainX.shape[0],1) + self.weight_constant
        result = (trainX*self.weight_first) + self.weight_constant
        '''for i in range(0,trainX.shape[1]-1):
            for j in range(i+1,trainX.shape[1]):
                result = result + np.dot(self.weight_second[i,:],self.weight_second[j,:])*(trainX[:,i]*trainX[:,j])'''
        return result
    
    def predict(self,trainX):
        return (1/(1+np.exp(-1*self.predict_raw(trainX))))
        
    def fit(self,TrainX,Trainy,k,ValX=None,Valy=None,verbose=False):
        ###init the model
        self.k = k
        trainX = self.data_type(data=TrainX)
        #trainyTmp = trainy.reshape(trainy.shape[0],1)
        trainyTmp = np.matrix(Trainy)
        self.weight_constant = 0
        self.weight_first = np.matrix(np.random.randn(trainX.shape[1])).T
        self.weight_second = np.matrix(np.random.randn(self.k,trainX.shape[1]))
        
        '''if batchSize <= 0:
            batchSize = trainXTmp.shape[0]'''
        for n in range(self.epochs):
            start_time = time.time()
            #gradientBase = -1*trainyTmp*np.exp(-1*trainyTmp*self.predict_raw(trainXTmp))/(1+np.exp(-1*trainyTmp*self.predict_raw(trainXTmp)))
            gradientBase = -1*trainyTmp + self.predict(trainX)
            #print(gradientBase)

            self.learningRate = 0.9*self.learningRate
            for i in range(trainX.shape[0]):
                #recordTmp = trainX[i].reshape(trainX.shape[1],1)
                recordTmp = trainX[i].T
                gradientBaseTmp = gradientBase[i]
                #gradientInteraction = (self.weight_second*recordTmp).sum(axis=0)
                print(self.weight_second.shape)
                print(gradientBaseTmp.shape)
                gradientInteraction = self.weight_second*recordTmp
                self.weight_constant = self.weight_constant - self.learningRate*gradientBaseTmp
                self.weight_first = self.weight_first - self.learningRate*gradientBaseTmp*recordTmp   
                #self.weight_second = self.weight_second - self.learningRate*gradientBaseTmp*(np.outer(recordTmp,gradientInteraction)-np.power(recordTmp,2)*self.weight_second)
                self.weight_second = self.weight_second - self.learningRate*gradientBaseTmp*(recordTmp*gradientInteraction.T-np.power(recordTmp,2)*self.weight_second)
            if verbose:
                print('epoch--%d in %.2f s'%(n,time.time()-start_time))
                print('Train Set accuracy: %.4f'%self.accuracy(trainX,trainy))
                try: print('Validation Set accuracy: %.4f'%self.accuracy(valX,valy))
                except: continue
            
    def accuracy(self,trainX,trainy):
        return -1*(np.log(1- trainy.reshape(trainy.shape[0],1)-self.predict(trainX)+2*trainy.reshape(trainy.shape[0],1)*self.predict(trainX)).mean())
                
            

In [20]:
fm =fm_tool(learningRate=0.01,epochs=100)
fm.fit(TrainX=np.random.randn(200).reshape(20,10),Trainy=np.array([0]*10+[1]*10),k=32,verbose=True)

(32, 10)
(1, 20)


ValueError: dimension mismatch

In [ ]:
fm.predict(np.random.randn(200).reshape(20,10))

In [ ]:
a = np.array([1]*100).reshape(10,10)

In [ ]:
type(a) is np.ndarray

In [ ]:
for i in range(a.shape[0]):
    a[i,:i+1] = 0
a

In [ ]:
b = np.matrix(a)
c = np.matrix(a+1)

In [ ]:
b = np.matrix(b)
b

In [ ]:
import numpy as np
from scipy import sparse
A = np.array([1,2,0])
B = np.matrix([[1,2,0],[0,0,3],[1,0,4]])

In [ ]:
sA = sparse.csr_matrix(A)
sB = sparse.csr_matrix(B)
sC =sB

In [ ]:
help(sp.sparse)

In [ ]:
sA.T*sA

In [ ]:
t.sum(axis=0)

In [ ]:
B*B

In [ ]:
B

In [ ]:
B+B

In [ ]:
fm.predict(np.random.randn(200).reshape(20,10))

In [ ]:
fm.recordTmp.shape

In [ ]:
fm.gradientBase[1].shape

In [ ]:
a

In [ ]:
b

In [ ]:
a = np.arange(200).reshape(20,10)
b = np.arange(10).reshape(10,1)

In [ ]:
(a.T*b).sum(axis=0).reshape(,1)

In [ ]:
b[0].reshape(2,1)

In [ ]:
a = np.random.randn(3,2)
a

In [ ]:
(a*b)+b

In [ ]:
''' Based on Tinrtgu's FTRL code: http://www.kaggle.com/c/avazu-ctr-prediction/forums/t/10927/beat-the-benchmark-with-less-than-1mb-of-memory
'''

from csv import DictReader
from math import exp, copysign, log, sqrt
from datetime import datetime
import random

class FM_FTRL_machine(object):
    
    def __init__(self, fm_dim, fm_initDev, L1, L2, L1_fm, L2_fm, D, alpha, beta, alpha_fm = .1, beta_fm = 1.0, dropoutRate = 1.0):
        ''' initialize the factorization machine.'''
        
        self.alpha = alpha              # learning rate parameter alpha
        self.beta = beta                # learning rate parameter beta
        self.L1 = L1                    # L1 regularizer for first order terms
        self.L2 = L2                    # L2 regularizer for first order terms
        self.alpha_fm = alpha_fm        # learning rate parameter alpha for factorization machine
        self.beta_fm = beta_fm          # learning rate parameter beta for factorization machine
        self.L1_fm = L1_fm              # L1 regularizer for factorization machine weights. Only use L1 after one epoch of training, because small initializations are needed for gradient.
        self.L2_fm = L2_fm              # L2 regularizer for factorization machine weights.
        self.fm_dim = fm_dim            # dimension of factorization.
        self.fm_initDev = fm_initDev    # standard deviation for random intitialization of factorization weights.
        self.dropoutRate = dropoutRate  # dropout rate (which is actually the inclusion rate), i.e. dropoutRate = .8 indicates a probability of .2 of dropping out a feature.
        
        self.D = D
        
        # model
        # n: squared sum of past gradients
        # z: weights
        # w: lazy weights
        
        # let index 0 be bias term to avoid collisions.
        self.n = [0.] * (D + 1) 
        self.z = [0.] * (D + 1)
        self.w = [0.] * (D + 1)
        
        self.n_fm = {}
        self.z_fm = {}
        self.w_fm = {}
    
        
    def init_fm(self, i):
        ''' initialize the factorization weight vector for variable i.
        '''
        if i not in self.n_fm:
            self.n_fm[i] = [0.] * self.fm_dim
            self.w_fm[i] = [0.] * self.fm_dim
            self.z_fm[i] = [0.] * self.fm_dim
            
            for k in range(self.fm_dim): 
                self.z_fm[i][k] = random.gauss(0., self.fm_initDev)
    
    def predict_raw(self, x):
        ''' predict the raw score prior to logit transformation.
        '''
        alpha = self.alpha
        beta = self.beta
        L1 = self.L1
        L2 = self.L2
        alpha_fm = self.alpha_fm
        beta_fm = self.beta_fm
        L1_fm = self.L1_fm
        L2_fm = self.L2_fm
        
        # first order weights model
        n = self.n
        z = self.z
        w = self.w
        
        # FM interaction model
        n_fm = self.n_fm
        z_fm = self.z_fm
        w_fm = self.w_fm
        
        raw_y = 0.
        
        # calculate the bias contribution
        for i in [0]:
            # no regularization for bias
            w[i] = (- z[i]) / ((beta + sqrt(n[i])) / alpha)
            
            raw_y += w[i]
        
        # calculate the first order contribution.
        for i in x:
            sign = -1. if z[i] < 0. else 1. # get sign of z[i]
            
            if sign * z[i] <= L1:
                w[i] = 0.
            else:
                w[i] = (sign * L1 - z[i]) / ((beta + sqrt(n[i])) / alpha + L2)
            
            raw_y += w[i]
        
        len_x = len(x)
        # calculate factorization machine contribution.
        for i in x:
            self.init_fm(i)
            for k in range(self.fm_dim):
                sign = -1. if z_fm[i][k] < 0. else 1.   # get the sign of z_fm[i][k]
                
                if sign * z_fm[i][k] <= L1_fm:
                    w_fm[i][k] = 0.
                else:
                    w_fm[i][k] = (sign * L1_fm - z_fm[i][k]) / ((beta_fm + sqrt(n_fm[i][k])) / alpha_fm + L2_fm)
        
        for i in range(len_x):
            for j in range(i + 1, len_x):
                for k in range(self.fm_dim):
                    raw_y += w_fm[x[i]][k] * w_fm[x[j]][k]
        
        return raw_y
    
    def predict(self, x):
        ''' predict the logit
        '''
        return 1. / (1. + exp(- max(min(self.predict_raw(x), 35.), -35.)))
    
    def dropout(self, x):
        ''' dropout variables in list x
        '''
        for i, var in enumerate(x):
            if random.random() > self.dropoutRate:
                del x[i]
    
    def dropoutThenPredict(self, x):
        ''' first dropout some variables and then predict the logit using the dropped out data.
        '''
        self.dropout(x)
        return self.predict(x)
    
    def predictWithDroppedOutModel(self, x):
        ''' predict using all data, using a model trained with dropout.
        '''
        return 1. / (1. + exp(- max(min(self.predict_raw(x) * self.dropoutRate, 35.), -35.)))
    
    def update(self, x, p, y):
        ''' Update the parameters using FTRL (Follow the Regularized Leader)
        '''
        alpha = self.alpha
        alpha_fm = self.alpha_fm
        
        # model
        n = self.n
        z = self.z
        w = self.w
        
        # FM model
        n_fm = self.n_fm
        z_fm = self.z_fm
        w_fm = self.w_fm
        
        # cost gradient with respect to raw prediction.
        g = p - y
        
        fm_sum = {}      # sums for calculating gradients for FM.
        len_x = len(x)
        
        for i in x + [0]:
            # update the first order weights.
            sigma = (sqrt(n[i] + g * g) - sqrt(n[i])) / alpha
            z[i] += g - sigma * w[i]
            n[i] += g * g
            
            # initialize the sum of the FM interaction weights.
            fm_sum[i] = [0.] * self.fm_dim
        
        # sum the gradients for FM interaction weights.
        for i in range(len_x):
            for j in range(len_x):
                if i != j:
                    for k in range(self.fm_dim):
                        fm_sum[x[i]][k] += w_fm[x[j]][k]
        
        for i in x:
            for k in range(self.fm_dim):
                g_fm = g * fm_sum[i][k]
                sigma = (sqrt(n_fm[i][k] + g_fm * g_fm) - sqrt(n_fm[i][k])) / alpha_fm
                z_fm[i][k] += g_fm - sigma * w_fm[i][k]
                n_fm[i][k] += g_fm * g_fm
    
    def write_w(self, filePath):
        ''' write out the first order weights w to a file.
        '''
        with open(filePath, "w") as f_out:
            for i, w in enumerate(self.w):
                f_out.write("%i,%f\n" % (i, w))
    
    def write_w_fm(self, filePath):
        ''' write out the factorization machine weights to a file.
        '''
        with open(filePath, "w") as f_out:
            for k, w_fm in self.w_fm.iteritems():
                f_out.write("%i,%s\n" % (k, ",".join([str(w) for w in w_fm])))


def logLoss(p, y):
    ''' 
    calculate the log loss cost
    p: prediction [0, 1]
    y: actual value {0, 1}
    '''
    p = max(min(p, 1. - 1e-15), 1e-15)
    return - log(p) if y == 1. else -log(1. - p)

def data(filePath, hashSize, hashSalt):
    ''' generator for data using hash trick
    
    INPUT:
        filePath
        hashSize
        hashSalt: String with which to salt the hash function
    '''
    
    for t, row in enumerate(DictReader(open(filePath))):
        ID = row['id']
        del row['id']
        
        y = 0.
        if 'click' in row:
            if row['click'] == '1':
                y = 1.
            del row['click']
        
        date = int(row['hour'][4:6])
        
        row['hour'] = row['hour'][6:]
        
        x = []
        
        for key in row:
            value = row[key]
            
            index = abs(hash(hashSalt + key + '_' + value)) % hashSize + 1      # 1 is added to hash index because I want 0 to indicate the bias term.
            x.append(index)
        
        yield t, date, ID, x, y